In [1]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## Data Loading and Visualization

In [2]:
# Read currency data
#merged_data.to_csv('Resources/Price-Data', index=False)

directory = 'Resources/Price-Data'
all_files = os.listdir(directory)
df_list = []
for file in all_files:
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, file))
        # Extract the first three letters of the filename
        file_prefix = file[:3]
        # Create a new column with the file prefix value for each row
        df['Index'] = file_prefix
        df_list.append(df)
merged_df = pd.concat(df_list)
merged_df

,Date,Open,High,Low,Close,Volume,Index
0,2003-12-01,3.672000,3.672000,3.671500,3.671500,0,AED
1,2003-12-02,3.670900,3.672000,3.670800,3.671500,0,AED
2,2003-12-03,3.671900,3.672000,3.671500,3.671500,0,AED
3,2003-12-04,3.671800,3.672000,3.671500,3.671500,0,AED
4,2003-12-05,3.671900,3.672000,3.671800,3.671800,0,AED
...,...,...,...,...,...,...,...
2839,2023-11-28,23.517496,23.572788,23.517496,23.533171,0,ZMW
2840,2023-11-29,23.468376,23.697779,23.468376,23.572788,0,ZMW
2841,2023-11-30,23.739799,23.739799,23.681280,23.697779,0,ZMW
2842,2023-12-01,23.698650,23.712244,23.698650,23.681280,0,ZMW


In [3]:
merged_df.to_csv('Resources/currency.csv', index=False)

In [4]:
# Create a scatter plot of Open versus the Close information
index_value = 'GBP'
# Filter the DataFrame for rows with the specific index value
df_currency = merged_df[merged_df['Index'] == index_value]
df_currency.head()
df_currency.hvplot.line(
    x='Date',
    y= 'Open',
    xlabel='Date',
    ylabel='Currency',
    title='Currency vs Time'
)

:Curve   [Date]   (Open)

## Data Preparation

In [5]:
# Convert the 'Date' column to datetime format
df_currency['Date'] = pd.to_datetime(df_currency['Date'])

# Calculate the number of days since the start date
start_date = df_currency['Date'].min()
df_currency['Days'] = (df_currency['Date'] - start_date).dt.days

# Reformat data of the independent variable X as a single-column array
X = df_currency['Days'].values.reshape(-1, 1)

# Display sample data
X[:5]

C:\Users\david\AppData\Local\Temp\ipykernel_3568\3833139901.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_currency['Date'] = pd.to_datetime(df_currency['Date'])
C:\Users\david\AppData\Local\Temp\ipykernel_3568\3833139901.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_currency['Days'] = (df_currency['Date'] - start_date).dt.days


array([[0],
       [1],
       [2],
       [3],
       [4]], dtype=int64)

In [6]:
# The shape of X is 30 samples, with a single feature (column)
X.shape

(5204, 1)

In [7]:
# Create an array for the dependent variable y
y = df_currency["Close"]

## Building the Linear Regression Model

In [8]:
# Create a model with scikit-learn
model = LinearRegression()

In [9]:
# Fit the data into the model
model.fit(X, y)

LinearRegression()

In [10]:
# Display the slope
print(f"Model's slope: {model.coef_}")

Model's slope: [4.27639798e-05]


In [11]:
# Display the y-intercept
print(f"Model's y-intercept: {model.intercept_}")

Model's y-intercept: 0.5080240620373949


In [12]:
# Display the model's best fit line formula
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")

Model's formula: y = 0.5080240620373949 + 4.27639798231102e-05X


In [13]:
# Predict the currency for a future date
future_date = pd.to_datetime('2024-01-01')  # Change this date to your desired future date
future_days = (future_date - start_date).days  # Calculate days since start date for the future date
predicted_currency = model.predict([[future_days]])  # Predict for the future date

# Display the prediction for the future date
print(f"Predicted closing currency on {future_date} is: {predicted_currency[0]:.2f}")

Predicted closing currency on 2024-01-01 00:00:00 is: 0.82


In [14]:
# Make predictions using the X set
predicted_y_values = model.predict(X)

In [15]:
# Create a copy of the original data
df_currency_predicted = df_currency.copy()

# Add a column with the predicted currency values
df_currency_predicted["currency_predicted"] = predicted_y_values

# Display sample data
df_currency_predicted.head()

,Date,Open,High,Low,Close,Volume,Index,Days,currency_predicted
0,2003-12-01,0.58011,0.58194,0.57897,0.58187,0,GBP,0,0.508024
1,2003-12-02,0.58173,0.58231,0.57733,0.57793,0,GBP,1,0.508067
2,2003-12-03,0.57787,0.57964,0.57743,0.57867,0,GBP,2,0.508110
3,2003-12-04,0.57890,0.58197,0.57837,0.58116,0,GBP,3,0.508152
4,2003-12-05,0.58119,0.58146,0.57700,0.57700,0,GBP,4,0.508195


In [16]:
# Create a line plot of open versus the predicted currency values
best_fit_line = df_currency_predicted.hvplot.line(
    x='Date',
    y='currency_predicted',
    xlabel='Date',
    ylabel='Predicted Currency',
    title='Predicted Currency Vs. Time'
)
best_fit_line

:Curve   [Date]   (currency_predicted)

In [17]:
# Superpose the original data and the best fit line
# Create a line plot of 'Open' versus the predicted currency values
df_currency_predicted.hvplot.line(
    x='Date',
    y=['Open', 'currency_predicted'], 
    xlabel='Date',
    ylabel='Currency',
    title='Currency vs Predicted Currency'
)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

## Linear Regression Model Assessment

In [18]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = model.score(X, y)
predicted_y_values = model.predict(X)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print relevant metrics
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.8033595406477351.
The r2 is 0.8033595406477351.
The mean squared error is 0.001993080545406878.
The root mean squared error is 0.04464393066707812.
The standard deviation is 0.10067600762261003.
